In [110]:
from sage.algebras.lie_algebras.bch import bch_iterator
import numpy as np
import re
import time 

In [111]:
d = 3
L = 4
Lbl = LieAlgebra(QQ, d, step=L)
B = len(Lbl.gens())

In [112]:
R = PolynomialRing(QQ, 2*B, 'm')
m = R.gens()

In [113]:
def rNF(p,V):
    if p == 0:
        return 0
    r = p
    for t in range(len(V)):
        if r.lm() == lcm(r.lm(), V[t].lm()):
            a = r.lm()//V[t].lm()
            if 1 == gcd(a,prod([m[i] for i in range(0,B)])):
                r = rNF(r - r.lc()/V[t].lc()*a*V[t],V)
        if r == 0:
            return 0
    return r.lt() + rNF(r - r.lt(), V)

In [114]:
def Buchberger(F):
    lenF = len(F)
    for i in range(len(F)):
        for j in range(i,len(F)): 
            lcm_gh = lcm(F[i].lm(),F[j].lm())
            bi = (lcm_gh == lcm(lcm_gh, F[i].lm()))    
            bj = (lcm_gh == lcm(lcm_gh, F[j].lm()))
            ai = lcm_gh//F[i].lm()
            aj = lcm_gh//F[j].lm()
            b2i = (1 == gcd(ai,prod([m[t] for t in range(0,B)])))
            b2j = (1 == gcd(aj,prod([m[t] for t in range(0,B)])))
            if (bi and bj and b2i and b2j):
                r = rNF(F[i]*ai/F[i].lc() - F[j]*aj/F[j].lc(),F)
                if not (r == 0):
                    F = F + [r/r.lc()]
    if not (lenF == len(F)):
        return Buchberger(F)
    return F

In [115]:
def interReduce(F):
    lenF = len(F) 
    for i in range(lenF):
        if (i < len(F)):
            F[i] = rNF(F[i],F[:i]+F[i+1:])
            F = list(filter(lambda k: not (k == 0),F))
    if lenF == len(F):
        return F
    return interReduce(F)

In [116]:
def poly2latex(_p):
    res = re.sub(r'()m([0-9]*)()', r'\1m_{\2}\3', str(_p))
    res = re.sub(r'()([0-9]*)/([0-9]*)()', r'\1frac{\2}{\3}\4', res)
    for i in range(2*B-1,B-1,-1):
        res = res.replace("m_{"+str(i),"M_{"+str(i+1-B))
    for i in range(B-1,-1,-1):
        res = res.replace("m_{"+str(i),"C_{"+str(i+1))
    res = res.replace("*","")
    return res

def poly2human(_p):
    res = str(_p)
    for i in range(2*B-1,B-1,-1):
        res = res.replace("m"+str(i),"M"+str(i+1-B))
    for i in range(B-1,-1,-1):
        res = res.replace("m"+str(i),"C"+str(i+1))
    return res

def poly2pythonEx(st_poly):
    res = re.sub(r'\^', '**',   str(st_poly))
    res = re.sub(r'()m([0-9]*)()', r'\1m[\2]\3', res)
    return "lambda m : "+res

In [117]:
LieAlg = LieAlgebra(PolynomialRing(QQ, 2*B, 'm'), d, step=L)
LieAlg.inject_variables()

logSXn = LieAlg.from_vector(vector(R, m[:B]))
y = -LieAlg.from_vector(vector(R, m[B:]))
S = sum([Z for Z in bch_iterator(y, logSXn)])
q = S.monomial_coefficients()
q = list(q.values())                # derive polynomials 

Defining X_1, X_2, X_3, X_12, X_13, X_23, X_112, X_113, X_122, X_123, X_132, X_133, X_223, X_233, X_1112, X_1113, X_1122, X_1123, X_1132, X_1133, X_1213, X_1222, X_1223, X_1232, X_1233, X_1322, X_1323, X_1332, X_1333, X_2223, X_2233, X_2333


In [118]:
poly2human(m[0]), poly2human(m[B]) ,poly2human(m[0]+m[1]+m[B]+m[0+B]*m[1+B])


('C1', 'M1', 'M1*M2 + C1 + C2 + M1')

In [119]:
[poly2human(qj) for qj in q]
#[poly2latex(qj) for qj in q]

['C1 - M1',
 'C2 - M2',
 'C3 - M3',
 '-1/2*C2*M1 + 1/2*C1*M2 + C4 - M4',
 '-1/2*C3*M1 + 1/2*C1*M3 + C5 - M5',
 '-1/2*C3*M2 + 1/2*C2*M3 + C6 - M6',
 '1/12*C1*C2*M1 + 1/12*C2*M1^2 - 1/12*C1^2*M2 - 1/12*C1*M1*M2 - 1/2*C4*M1 + 1/2*C1*M4 + C7 - M7',
 '1/12*C1*C3*M1 + 1/12*C3*M1^2 - 1/12*C1^2*M3 - 1/12*C1*M1*M3 - 1/2*C5*M1 + 1/2*C1*M5 + C8 - M8',
 '-1/12*C2^2*M1 + 1/12*C1*C2*M2 - 1/12*C2*M1*M2 + 1/12*C1*M2^2 + 1/2*C4*M2 - 1/2*C2*M4 + C9 - M9',
 '-1/12*C2*C3*M1 + 1/6*C1*C3*M2 + 1/12*C3*M1*M2 - 1/12*C1*C2*M3 - 1/6*C2*M1*M3 + 1/12*C1*M2*M3 - 1/2*C6*M1 + 1/2*C4*M3 - 1/2*C3*M4 + 1/2*C1*M6 + C10 - M10',
 '-1/6*C2*C3*M1 + 1/12*C1*C3*M2 - 1/12*C3*M1*M2 + 1/12*C1*C2*M3 - 1/12*C2*M1*M3 + 1/6*C1*M2*M3 + 1/2*C5*M2 + 1/2*C4*M3 - 1/2*C3*M4 - 1/2*C2*M5 + C11 - M11',
 '-1/12*C3^2*M1 + 1/12*C1*C3*M3 - 1/12*C3*M1*M3 + 1/12*C1*M3^2 + 1/2*C5*M3 - 1/2*C3*M5 + C12 - M12',
 '1/12*C2*C3*M2 + 1/12*C3*M2^2 - 1/12*C2^2*M3 - 1/12*C2*M2*M3 - 1/2*C6*M2 + 1/2*C2*M6 + C13 - M13',
 '-1/12*C3^2*M2 + 1/12*C2*C3*M3 - 1/12*C3*M

In [124]:
h1_19 = interReduce(q[:19])
print(len(h1_19))
print(poly2human(h1_19[17]))
print(poly2human(h1_19[18]))
[poly2human(qj) for qj in h1_19]

19
-1/12*C3*C4*M1 + 1/12*C1*C6*M1 - 1/12*C1*C4*M3 + 1/6*C1*C3*M4 - 1/12*C1^2*M6 - 2/3*C10*M1 + 1/3*C11*M1 + C8*M2 - C7*M3 + M3*M7 - M2*M8 + 2/3*M1*M10 - 1/3*M1*M11 + C18 - M18
-1/12*C2*C5*M1 - 1/12*C1*C6*M1 - 1/12*C1*C5*M2 + 1/6*C1*C2*M5 + 1/12*C1^2*M6 + 1/3*C10*M1 - 1/6*C11*M1 - 1/2*C8*M2 + 1/2*C7*M3 - 1/2*M3*M7 + 1/2*M2*M8 - 1/3*M1*M10 + 1/6*M1*M11 - C18 + C19 + M18 - M19


['C1 - M1',
 'C2 - M2',
 'C3 - M3',
 'C4 - M4',
 'C5 - M5',
 'C6 - M6',
 '1/12*C1*C2*M1 - 1/12*C1^2*M2 + C7 - M7',
 '1/12*C1*C3*M1 - 1/12*C1^2*M3 + C8 - M8',
 '-1/12*C2^2*M1 + 1/12*C1*C2*M2 + C9 - M9',
 '1/8*C1*C3*M2 - 1/8*C1*C2*M3 + C10 - 1/2*C11 - M10 + 1/2*M11',
 '-1/6*C2*C3*M1 + 1/6*C1*C2*M3 - 2/3*C10 + 4/3*C11 + 2/3*M10 - 4/3*M11',
 '-1/12*C3^2*M1 + 1/12*C1*C3*M3 + C12 - M12',
 '1/12*C2*C3*M2 - 1/12*C2^2*M3 + C13 - M13',
 '-1/12*C3^2*M2 + 1/12*C2*C3*M3 + C14 - M14',
 '1/12*C1*C4*M1 - 1/12*C1^2*M4 + C15 - M15',
 '1/12*C1*C5*M1 - 1/12*C1^2*M5 + C16 - M16',
 '-1/12*C2*C4*M1 - 1/12*C1*C4*M2 + 1/6*C1*C2*M4 + C17 - M17',
 '-1/12*C3*C4*M1 + 1/12*C1*C6*M1 - 1/12*C1*C4*M3 + 1/6*C1*C3*M4 - 1/12*C1^2*M6 - 2/3*C10*M1 + 1/3*C11*M1 + C8*M2 - C7*M3 + M3*M7 - M2*M8 + 2/3*M1*M10 - 1/3*M1*M11 + C18 - M18',
 '-1/12*C2*C5*M1 - 1/12*C1*C6*M1 - 1/12*C1*C5*M2 + 1/6*C1*C2*M5 + 1/12*C1^2*M6 + 1/3*C10*M1 - 1/6*C11*M1 - 1/2*C8*M2 + 1/2*C7*M3 - 1/2*M3*M7 + 1/2*M2*M8 - 1/3*M1*M10 + 1/6*M1*M11 - C18 + C19 + M1

In [128]:
G19 = Buchberger(h1_19)
print(len(G19))
g19 = G19[18]
print(poly2human(g19))
[poly2human(qj) for qj in G19]

22
-1/12*C2*C5*M1 - 1/12*C1*C6*M1 - 1/12*C1*C5*M2 + 1/6*C1*C2*M5 + 1/12*C1^2*M6 + 1/3*C10*M1 - 1/6*C11*M1 - 1/2*C8*M2 + 1/2*C7*M3 - 1/2*M3*M7 + 1/2*M2*M8 - 1/3*M1*M10 + 1/6*M1*M11 - C18 + C19 + M18 - M19


['C1 - M1',
 'C2 - M2',
 'C3 - M3',
 'C4 - M4',
 'C5 - M5',
 'C6 - M6',
 '1/12*C1*C2*M1 - 1/12*C1^2*M2 + C7 - M7',
 '1/12*C1*C3*M1 - 1/12*C1^2*M3 + C8 - M8',
 '-1/12*C2^2*M1 + 1/12*C1*C2*M2 + C9 - M9',
 '1/8*C1*C3*M2 - 1/8*C1*C2*M3 + C10 - 1/2*C11 - M10 + 1/2*M11',
 '-1/6*C2*C3*M1 + 1/6*C1*C2*M3 - 2/3*C10 + 4/3*C11 + 2/3*M10 - 4/3*M11',
 '-1/12*C3^2*M1 + 1/12*C1*C3*M3 + C12 - M12',
 '1/12*C2*C3*M2 - 1/12*C2^2*M3 + C13 - M13',
 '-1/12*C3^2*M2 + 1/12*C2*C3*M3 + C14 - M14',
 '1/12*C1*C4*M1 - 1/12*C1^2*M4 + C15 - M15',
 '1/12*C1*C5*M1 - 1/12*C1^2*M5 + C16 - M16',
 '-1/12*C2*C4*M1 - 1/12*C1*C4*M2 + 1/6*C1*C2*M4 + C17 - M17',
 '-1/12*C3*C4*M1 + 1/12*C1*C6*M1 - 1/12*C1*C4*M3 + 1/6*C1*C3*M4 - 1/12*C1^2*M6 - 2/3*C10*M1 + 1/3*C11*M1 + C8*M2 - C7*M3 + M3*M7 - M2*M8 + 2/3*M1*M10 - 1/3*M1*M11 + C18 - M18',
 '-1/12*C2*C5*M1 - 1/12*C1*C6*M1 - 1/12*C1*C5*M2 + 1/6*C1*C2*M5 + 1/12*C1^2*M6 + 1/3*C10*M1 - 1/6*C11*M1 - 1/2*C8*M2 + 1/2*C7*M3 - 1/2*M3*M7 + 1/2*M2*M8 - 1/3*M1*M10 + 1/6*M1*M11 - C18 + C19 + M1

In [130]:
G17 = Buchberger(q[:17])
q19 = q[18]
s19 = rNF(q19,G17)
poly2human(s19)

'-1/12*C3*C4*M1 - 1/12*C2*C5*M1 - 1/12*C1*C5*M2 - 1/12*C1*C4*M3 + 1/6*C1*C3*M4 + 1/6*C1*C2*M5 + C19 - M19'

In [38]:
max_cand = 0
G = [[]]+[[q[t]] for t in range(B-1)]
s = [(rNF(q[i],G[i])) for i in range(B)]
for j in range(1,B):
    print("compute "+str(j)+" of "+str(B)+" GBs with length "+str(len(G[j-1])))
    G[j] = interReduce(Buchberger(G[j-1]+G[j]))
    #save(G[j],path+"/G"+str(j))
    #print("find better r"+str(j))
    len_cand = s[j].number_of_terms()
    for i in range(j+1):
        cand = rNF(q[j],G[i])
        if cand.number_of_terms() <= len_cand:
            s[j] = cand
            len_cand = cand.number_of_terms()
         #save(r[j],path+"/r"+str(j))
    if s[j].number_of_terms() > max_cand:
        max_cand = s[j].number_of_terms()
        print("new maximial number of terms in sj")
        print(max_cand)

compute 1 of 32 GBs with length 0
new maximial number of terms in sj
2
compute 2 of 32 GBs with length 1
compute 3 of 32 GBs with length 2
compute 4 of 32 GBs with length 3
compute 5 of 32 GBs with length 4
compute 6 of 32 GBs with length 5
new maximial number of terms in sj
4
compute 7 of 32 GBs with length 6
compute 8 of 32 GBs with length 7
compute 9 of 32 GBs with length 8
new maximial number of terms in sj
5
compute 10 of 32 GBs with length 9
compute 11 of 32 GBs with length 10
compute 12 of 32 GBs with length 12
compute 13 of 32 GBs with length 13
compute 14 of 32 GBs with length 15
compute 15 of 32 GBs with length 17
compute 16 of 32 GBs with length 18
compute 17 of 32 GBs with length 19
new maximial number of terms in sj
7
compute 18 of 32 GBs with length 20
new maximial number of terms in sj
8
compute 19 of 32 GBs with length 21
compute 20 of 32 GBs with length 22
compute 21 of 32 GBs with length 23
compute 22 of 32 GBs with length 24
compute 23 of 32 GBs with length 27
comput

In [39]:
r = [s[j] - m[j] + m[j+B] for j in range(B)]
print('average number of terms in rj')
print(sum([rj.number_of_terms() for rj in r])/B)
print('maximal number of terms in rj')
print(max([rj.number_of_terms() for rj in r]))

average number of terms in rj
2.78125
maximal number of terms in rj
7


In [84]:
# table SM4
print([rj.number_of_terms() for rj in r])

[0, 0, 0, 0, 0, 0, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 3, 5, 6, 3, 6, 2, 6, 5, 5, 5, 7, 5, 2, 2, 3, 2]


In [55]:
[poly2human(rj) for rj in r]

['0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1/12*C1*C2*M1 - 1/12*C1^2*M2',
 '1/12*C1*C3*M1 - 1/12*C1^2*M3',
 '-1/12*C2^2*M1 + 1/12*C1*C2*M2',
 '-1/12*C2*C3*M1 + 1/6*C1*C3*M2 - 1/12*C1*C2*M3',
 '-1/6*C2*C3*M1 + 1/12*C1*C3*M2 + 1/12*C1*C2*M3',
 '-1/12*C3^2*M1 + 1/12*C1*C3*M3',
 '1/12*C2*C3*M2 - 1/12*C2^2*M3',
 '-1/12*C3^2*M2 + 1/12*C2*C3*M3',
 '1/12*C1*C4*M1 - 1/12*C1^2*M4',
 '1/12*C1*C5*M1 - 1/12*C1^2*M5',
 '-1/12*C2*C4*M1 - 1/12*C1*C4*M2 + 1/6*C1*C2*M4',
 '-1/12*C3*C4*M1 + 1/12*C1*C6*M1 - 1/12*C1*C4*M3 + 1/6*C1*C3*M4 - 1/12*C1^2*M6',
 '-1/12*C3*C4*M1 - 1/12*C2*C5*M1 - 1/12*C1*C5*M2 - 1/12*C1*C4*M3 + 1/6*C1*C3*M4 + 1/6*C1*C2*M5',
 '-1/12*C3*C5*M1 - 1/12*C1*C5*M3 + 1/6*C1*C3*M5',
 '1/6*C3*C4*M1 - 1/6*C2*C5*M1 + 1/12*C1*C5*M2 - 1/12*C1*C4*M3 - 1/12*C1*C3*M4 + 1/12*C1*C2*M5',
 '1/12*C2*C4*M2 - 1/12*C2^2*M4',
 '-1/12*C2*C6*M1 + 1/6*C3*C4*M2 + 1/6*C1*C6*M2 - 1/12*C2*C4*M3 - 1/12*C2*C3*M4 - 1/12*C1*C2*M6',
 '-1/6*C2*C6*M1 + 1/4*C3*C4*M2 + 1/12*C1*C6*M2 - 1/4*C2*C3*M4 + 1/12*C1*C2*M6',
 '-1/12*C3*C6

In [71]:
######## numeric tests ###########
logSXn2 = LieAlg.from_vector(vector(R, m[:B]))
y2 = LieAlg.from_vector(vector(R, m[B:]))
S2 = sum([Z for Z in bch_iterator(y2, logSXn2)])
p = S2.monomial_coefficients()
p = list(p.values())                # derive polynomials p_j
for i in range(len(p)):
    p[i] -= m[i]
    p[i] -= m[i+B] 
    
logSXn = LieAlg.from_vector(vector(R, m[:B]))
y = -LieAlg.from_vector(vector(R, m[B:]))
S = sum([Z for Z in bch_iterator(y, logSXn)])
q = S.monomial_coefficients()
q = list(q.values())                # derive polynomials q_j
for i in range(len(q)):
    q[i] -= m[i]
    q[i] += m[i+B] 
    
r_npEx = [poly2pythonEx(rj) for rj in r]
p_npEx = [poly2pythonEx(pj) for pj in p]
via_q_npEx = [poly2pythonEx(qj) for qj in q]
outp = [eval(a) for a in r_npEx]
outp_p = [eval(a) for a in p_npEx]
outp_via_q = [eval(a) for a in via_q_npEx]
N = 1000
logSX = np.random.rand(N,B)


start_time = time.time()
res = np.zeros(B)
for k in range(B):
    res[k] = np.sum(logSX[:,k]+outp[k]([logSX[:,_k] for _k in range(B) ]+list(res)))/N
print(time.time() - start_time)
print(res)

start_time = time.time()
res = np.zeros(B)
for k in range(B):  # -res by purpouse, since p takes -m as an argument
    res[k] = np.sum(logSX[:,k]+outp_p[k]([logSX[:,_k] for _k in range(B) ]+list(-res)))/N
print(time.time() - start_time)
print(res)

start_time = time.time()
res = np.zeros(B)
for k in range(B):
    res[k] = np.sum(logSX[:,k]+outp_via_q[k]([logSX[:,_k] for _k in range(B) ]+list(res)))/N
print(time.time() - start_time)
print(res)

0.0021600723266601562
[0.51663156 0.48501628 0.50123477 0.48988337 0.48999265 0.50465917
 0.49185825 0.46765018 0.50871959 0.50144199 0.51199485 0.50489064
 0.48136614 0.50799463 0.50233855 0.49754174 0.49095934 0.504977
 0.49320371 0.49524966 0.49669865 0.50286385 0.50182596 0.50686711
 0.48353729 0.49164931 0.49628444 0.48618765 0.48806226 0.49467334
 0.51620002 0.50371778]
0.004094362258911133
[0.51663156 0.48501628 0.50123477 0.48988337 0.48999265 0.50465917
 0.49185825 0.46765018 0.50871959 0.50144199 0.51199485 0.50489064
 0.48136614 0.50799463 0.50233855 0.49754174 0.49095934 0.504977
 0.49320371 0.49524966 0.49669865 0.50286385 0.50182596 0.50686711
 0.48353729 0.49164931 0.49628444 0.48618765 0.48806226 0.49467334
 0.51620002 0.50371778]
0.009208440780639648
[0.51663156 0.48501628 0.50123477 0.48988337 0.48999265 0.50465917
 0.49185825 0.46765018 0.50871959 0.50144199 0.51199485 0.50489064
 0.48136614 0.50799463 0.50233855 0.49754174 0.49095934 0.504977
 0.49320371 0.49524966 

In [106]:
# Compare with antisymmetrized BCH formula
Ltemp = LieAlgebra(QQ,['B','C'])
print(Ltemp)
Lyn = Ltemp.Lyndon()
B,C = Lyn.gens()
bch = bch_iterator(B,C)
Stemp = Ltemp();
for i in range(floor((L + 1)/ 2)) :
    next(bch)
    T3list = next(bch).list()
    Stemp  = Stemp + sum([Lyn({item[0][1]: 2*item[1]}) for item in T3list])

def apply_bracket(bracket, Bsub, Csub, Liealg) :
    if (str(bracket) == 'B'): return Bsub
    if (str(bracket) == 'C'): return Csub
    return Liealg.bracket(apply_bracket(bracket[0], Bsub,Csub, Liealg), apply_bracket(bracket[1], Bsub,Csub, Liealg))

Snew = sum([item[1] * (apply_bracket(item[0],y,logSXn,LieAlg)) for item in Stemp.list()])
SvalNew = list(Snew.monomial_coefficients().values())

Free Lie algebra generated by (B, C) over Rational Field


In [107]:
[qj.number_of_terms() -1 for qj in SvalNew]

[0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 3,
 5,
 6,
 3,
 6,
 2,
 6,
 5,
 5,
 5,
 8,
 5,
 2,
 2,
 3,
 2]